In [1]:
import pandas as pd 
import sqlite3 as sql
from datetime import datetime 
import seaborn as sns

In [2]:
# Import CSV file in Google Colab 
from google.colab import files
uploaded = files.upload()

import io
WQ_Big_Lake_Sammamish_df = pd.read_csv(io.BytesIO(uploaded['Big_Lake_Sammamish.csv']))
WQ_Big_Lake_Sammamish_df.head(5)

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# Drop Unnamed column 
WQ_Big_Lake_Sammamish_df.drop('Unnamed: 0',axis=1, inplace=True)
WQ_Big_Lake_Sammamish_df

In [ ]:
#Rename Ammmonia_Nitrogen to Ammonia_Nitrogen
WQ_Big_Lake_Sammamish_df.rename({'Ammmonia Nitrogen (mg/L)':'Ammonia Nitrogen (mg/L)'},axis='columns',inplace=True)
WQ_Big_Lake_Sammamish_df.head()

In [ ]:
# Data Summary
WQ_Big_Lake_Sammamish_df.describe()

In [ ]:
# convert string to datetime
WQ_Big_Lake_Sammamish_df["CollectDate"] = pd.to_datetime(WQ_Big_Lake_Sammamish_df["CollectDate"])

In [ ]:
# Histogram shows the number of instances that have a given value range. Using jupyter's magic command to specify which backend Matplotlib should use.
%matplotlib inline
import matplotlib.pyplot as plt
WQ_Big_Lake_Sammamish_df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
# Visualize the value counts of target variable
WQ_Big_Lake_Sammamish_df["WQI"].plot.density()

In [ ]:
# Group by Collect Date 
WQ_Big_Lake_Sammamish_df_mean = WQ_Big_Lake_Sammamish_df.groupby(["CollectDate"]).mean()
WQ_Big_Lake_Sammamish_df_mean.head(5)

In [ ]:
sns.heatmap(WQ_Big_Lake_Sammamish_df_mean.corr(), annot=True)

In [ ]:
# correlation among the features of each features in the data frame.
corr_matrix = WQ_Big_Lake_Sammamish_df_mean.corr()
round(corr_matrix,2)

In [ ]:
# Looking for the correlations between WQI and rest of the features
corr_matrix =WQ_Big_Lake_Sammamish_df_mean.corr()
round(corr_matrix["WQI"],2).sort_values(ascending=False)

In [ ]:
# Scatter plots for each feature with Water Quality Index "WQI
fig,axis=plt.subplots(3,3)
axis[0,0].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Nitrate Nitrite (mg/L)"], s=1) 
axis[0, 0].set_title('Nitrate Nitrite')
axis[0,1].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Total Nitrogen (mg/L)"], s=1)
axis[0, 1].set_title('Total Nitrogen')
axis[0,2].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Dissolved Oxygen (mg/L)"], s=1)
axis[0, 2].set_title('Dissolved Oxygen')
axis[1,0].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Orthophosphate Phosphate (mg/L)"], s=1)
axis[1,0].set_title('Orthophosphate Phosphate')
axis[1,1].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Total Phosphorus (mg/L)"], s=1)
axis[1,1].set_title('Total Phosphorus')
axis[1,2].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Fecal Coliform (CFU/100ml)"], s=1)
axis[1,2].set_title('Fecal Coliform')
axis[2,0].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["pH"], s=1)
axis[2,0].set_title('pH')
axis[2,1].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Ammonia Nitrogen (mg/L)"], s=1)
axis[2,1].set_title('Ammonia Nitrogen')
axis[2,2].scatter(WQ_Big_Lake_Sammamish_df_mean["WQI"],WQ_Big_Lake_Sammamish_df_mean["Conductivity (µmhos/cm)"], s=1)
axis[2,2].set_title('Conductivity')
plt.tight_layout()
plt.show()

In [ ]:
WQ_Big_Lake_Sammamish_df_mean.plot(kind="scatter", x="Temperature (°C)", y="WQI")

In [ ]:
WQ_Big_Lake_Sammamish_df.dtypes

In [ ]:
WQI_df_1 = WQ_Big_Lake_Sammamish_df.drop(columns=["CollectDate"])
WQI_df_1.head()

In [ ]:
WQI_df_1["WQI"] = round(WQI_df_1["WQI"],2)

In [ ]:
#Create the database
conn = sql.connect('King_County_WQI.db')
c = conn.cursor()

In [ ]:
#Table of Parameters for King County Big Lakes 
create_sql="CREATE TABLE IF NOT EXISTS Big_Lake_Sammamish (collectdate INTEGER, Ammonia_Nitrogen INTEGER, Conductivity INTEGER, Dissolved_Oxygen INTEGER, Fecal_Coliform INTEGER, Nitrate_Nitrite INTEGER, Orthophosphate_Phosphate INTEGER, pH INTEGER, Temperature INTEGER, Total_Nitrogen INTEGER, Total_Phosphorous INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
#DataFrame all at once 
WQ_Big_Lake_Sammamish_df_mean.to_sql(name="Big_Lake_Sammamish", con=conn, if_exists="replace",index=False)
WQ_Big_Lake_Sammamish_df_mean.head()

In [ ]:
#Table of Parameters for CollectDate Vs WQI  
create_sql="CREATE TABLE IF NOT EXISTS CollectDate_Vs_WQI  (collectdate INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
from tables.table import Column
#Row by row method 
for row in WQ_Big_Lake_Sammamish_df_mean.itertuples():
    insert_sql=f"INSERT INTO CollectDate_Vs_WQI (collectdate,WQI) VALUES('{row[0]}','{row[11]}')"
    cursor.execute(insert_sql) 

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import sklearn as skl
import tensorflow as tf

In [ ]:
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler

X = WQI_df_1.drop(["WQI"],1).values
y = WQI_df_1["WQI"].values.reshape(-1,1)


# To check the number of shapes
print(X.shape)
print(y.shape)

# To check the first five entries
print(y[:5])

In [ ]:
# Create a StandardScaler instances
scaler_X = skl.preprocessing.StandardScaler()
scaler_y = skl.preprocessing.StandardScaler()

# Use sklearn to split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


# Fit the scaler
scaler_X.fit(X_train)
scaler_y.fit(y_train)

# Scale the data
X_train_scaled = scaler_X.transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
# y_train_scaled = scaler_y.transform(y_train)
# y_test_scaled = scaler_y.transform(y_test)
y_train
y_test

print(X_train_scaled.shape)
print(y_train.shape)
print(X_test_scaled.shape)
print(y_test.shape)

In [ ]:
# importing the libraries
from keras.models import Sequential
from keras.layers import Dense
 
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = 10
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 15

model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))


# Second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="linear"))


# Check the structure of the model
model.summary()

In [ ]:
# Compiling the model
#model.compile(loss='mean_squared_error', optimizer="adam", metrics=["mse"])
model.compile(loss='mean_squared_error', optimizer="adam")

In [ ]:
# Fitting the ANN to the Training set
history = model.fit(X_train_scaled,y_train,validation_data=(X_test_scaled, y_test), epochs=100)

In [ ]:
# Making predictions using the testing data.
y_pred = model.predict(X_test_scaled)
print(y_pred.shape)
print(y_pred)

In [ ]:
print(y_test)

In [ ]:
y_pred_new = []
for i in y_pred:
    y_pred_new.append(i[0])
    
y_test_new = []
for i in y_test:
    y_test_new.append(i[0])

In [ ]:
import pandas as pd

accuracy_df = pd.DataFrame({"Prediction":y_pred_new, "Actual":y_test_new})
accuracy_df.head(20)

In [ ]:
#Table for WQI Predictions_Vs_Actual 
create_sql= "CREATE TABLE IF NOT EXISTS Predictions_Vs_Actual ('Prediction' INTEGER, 'Actual' INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
#DataFrame all at once 
accuracy_df.to_sql(name="Predictions_Vs_Actual ", con=conn, if_exists="replace",index=False)
accuracy_df.head(10)

In [ ]:
from tables.table import Column
#Row by row method 
for row in accuracy_df.itertuples():
   
    insert_sql=f"INSERT INTO Predictions_Vs_Actual (Prediction,Actual) VALUES('{row[1]}','{row[2]}')"
    cursor.execute(insert_sql) 

In [ ]:
# WQI Predictions_Vs_Actual in the Excellent Range 
cursor=c.execute('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 91 and 100
ORDER BY Prediction;''')
for row in cursor:
    print(row)

In [ ]:
# Create DataFrame Predictions_Vs_Actual in the Excellent Range for Lake Sammamish 
Predictions_Vs_Actual_Excellent_Lake_Sammamish_df =pd.read_sql('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 91 and 100
ORDER BY Prediction;''',conn)

Predictions_Vs_Actual_Excellent_Lake_Sammamish_df.head(5)

In [ ]:
#Table of Parameters Predictions_Vs_Actual in the Excellent Range for Lake Sammamish 
create_sql="CREATE TABLE IF NOT EXISTS Predictions_Vs_Actual_Excellent_Lake_Sammamish (collectdate INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
from tables.table import Column
#Row by row method 
for row in Predictions_Vs_Actual_Excellent_Lake_Sammamish_df.itertuples():
    insert_sql=f"INSERT INTO Predictions_Vs_Actual_Excellent_Lake_Sammamish(collectdate,WQI) VALUES('{row[1]}','{row[2]}')"
    cursor.execute(insert_sql) 

In [ ]:
# WQI Predictions_Vs_Actual in the Good Range 
cursor=c.execute('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 76 and 90
ORDER BY Prediction;''')
for row in cursor:
    print(row)

In [ ]:
# Create DataFrame Predictions_Vs_Actual in the Good Range for Lake Sammamish 
Predictions_Vs_Actual_Good_Lake_Sammamish_df =pd.read_sql('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 76 and 90
ORDER BY Prediction;''',conn)

Predictions_Vs_Actual_Good_Lake_Sammamish_df.head(5)

In [ ]:
#Table of Parameters Predictions_Vs_Actual in the Good Range for Lake Sammamish 
create_sql="CREATE TABLE IF NOT EXISTS Predictions_Vs_Actual_Good_Lake_Sammamish(collectdate INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
from tables.table import Column
#Row by row method 
for row in Predictions_Vs_Actual_Good_Lake_Sammamish_df.itertuples():
    insert_sql=f"INSERT INTO Predictions_Vs_Actual_Good_Lake_Sammamish(collectdate,WQI) VALUES('{row[1]}','{row[2]}')"
    cursor.execute(insert_sql) 

In [ ]:
# WQI Predictions_Vs_Actual in the Fair Range 
cursor=c.execute('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 51 and 75
ORDER BY Prediction;''')
for row in cursor:
    print(row)

In [ ]:
# Create DataFrame for WQI in the Predictions_Vs_Actual Fair Range Lake Sammamish 
Predictions_Vs_Actual_Fair_Lake_Sammamish_df =pd.read_sql('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 51 and 75
ORDER BY Prediction;''',conn)

Predictions_Vs_Actual_Fair_Lake_Sammamish_df.head(5)

In [ ]:
#Table of Parameters for WQI in the Predictions_Vs_Actual  Fair Range Lake Sammamish 
create_sql="CREATE TABLE IF NOT EXISTS Predictions_Vs_Actual_Fair_Lake_Sammamish (collectdate INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
from tables.table import Column
#Row by row method 
for row in Predictions_Vs_Actual_Fair_Lake_Sammamish_df.itertuples():
    insert_sql=f"INSERT INTO Predictions_Vs_Actual_Fair_Lake_Sammamish (collectdate,WQI) VALUES('{row[1]}','{row[2]}')"
    cursor.execute(insert_sql) 

In [ ]:
# WQI Predictions_Vs_Actual in the Poor Range 
cursor=c.execute('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 26 and 50
ORDER BY Prediction;''')
for row in cursor:
    print(row)

In [ ]:
# Create DataFrame for WQI in the Predictions_Vs_Actual Poor Range Lake Sammamish 
Predictions_Vs_Actual_Poor_Lake_Sammamish_df =pd.read_sql('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 26 and 50
ORDER BY Prediction;''',conn)

Predictions_Vs_Actual_Poor_Lake_Sammamish_df.head(5)

In [ ]:
#Table of Parameters for WQI in the Predictions_Vs_Actual Poor Range Lake Sammamish 
create_sql="CREATE TABLE IF NOT EXISTS Predictions_Vs_Actual_Poor_Lake_Sammamish(collectdate INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
from tables.table import Column
#Row by row method 
for row in Predictions_Vs_Actual_Poor_Lake_Sammamish_df.itertuples():
    insert_sql=f"INSERT INTO Predictions_Vs_Actual_Poor_Lake_Sammamish(collectdate,WQI) VALUES('{row[1]}','{row[2]}')"
    cursor.execute(insert_sql) 

In [ ]:
# WQI Predictions_Vs_Actual in the Very Poor Range 
cursor=c.execute('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 0 and 25
ORDER BY Prediction;''')
for row in cursor:
    print(row)

In [ ]:
# WQI Predictions_Vs_Actual in the Very Poor Range 
Predictions_Vs_Actual_Very_Poor_Lake_Sammamish_df=pd.read_sql('''
SELECT Prediction,Actual
FROM Predictions_Vs_Actual
WHERE Prediction BETWEEN 0 and 25
ORDER BY Prediction;''',conn)
Predictions_Vs_Actual_Very_Poor_Lake_Sammamish_df.head(5)

In [ ]:
#Table of Parameters for WQI in the Very Poor Range Lake Sammamish 
create_sql="CREATE TABLE IF NOT EXISTS Predictions_Vs_Actual_Very_Poor_Lake_Sammamish(collectdate INTEGER, WQI INTEGER)"
cursor=conn.cursor()
cursor.execute(create_sql)

In [ ]:
# INSERTING VALUES
from tables.table import Column
#Row by row method 
for row in Predictions_Vs_Actual_Very_Poor_Lake_Sammamish_df.itertuples():
    insert_sql=f"INSERT INTO Predictions_Vs_Actual_Very_Poor_Lake_Sammamish(collectdate,WQI) VALUES('{row[1]}','{row[2]}')"
    cursor.execute(insert_sql) 

In [ ]:
# evaluate the model
train_mse = model.evaluate(X_train_scaled, y_train, verbose=0)
test_mse = model.evaluate(X_test_scaled,y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_mse, test_mse))

In [ ]:
from matplotlib import pyplot
#plot loss during training
pyplot.title('Loss / Mean Squared Error')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
#pyplot.plot(history.history['mean_squared_error'], label='mse')
pyplot.legend()
pyplot.show()

In [ ]:
from matplotlib import pyplot
plt.figure(figsize=(10, 5))
plt.title('')
plt.plot(accuracy_df['Actual'], label='Actual values', color='orange', linewidth=5)
plt.plot(accuracy_df['Prediction'], label='Predicted', linewidth=1, linestyle='dashed')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot
plt.title('')
#plt.plot(accuracy_df['Actual'], label='Actual values')
plt.plot(accuracy_df['Prediction'], label='Predicted')
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot
plt.title('')
x=accuracy_df['Actual']
y=accuracy_df['Prediction']
plt.scatter(x,y)

plt.legend()
plt.show()

In [ ]:
# Close database connection
conn.commit()
conn.close()